# 🆔 マスターIDリスト取得ツール (Web Crawler)
Netkeibaの開催カレンダーを巡回し、指定期間内の**全てのレースID**を取得して `race_ids.csv` (JRA) / `race_ids_nar.csv` (NAR) を作成します。

これらが「マスターリスト」となり、スクレイピングツールはこのリストにあるが手元のCSVにないデータのみを取得するようになります。

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🛡️ Keep-Alive (アイドルタイムアウト回避)
from IPython.display import display, Javascript

display(Javascript('''
function ClickConnect(){
    console.log("Keep-alive: Working");
    var buttons = document.querySelectorAll("colab-connect-button");
    buttons.forEach(function(btn){
        btn.click();
    });
}
setInterval(ClickConnect, 60000);
console.log("Keep-alive script started - clicks every 60 seconds");
'''))

print("✅ Keep-alive activated (auto-clicks every 60 seconds)")
print("💡 This prevents idle timeout during long scraping sessions")

In [ ]:
# 設定
DATA_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
JRA_ID_CSV = 'race_ids.csv'
NAR_ID_CSV = 'race_ids_nar.csv'

START_YEAR = 2025
END_YEAR = 2026

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
from tqdm.auto import tqdm

def get_kaisai_dates(year, month, mode='JRA'):
    """カレンダーページから開催日を取得"""
    base_domain = "race.netkeiba.com" if mode == 'JRA' else "nar.netkeiba.com"
    cal_url = f"https://{base_domain}/top/calendar.html?year={year}&month={month}"
    
    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    
    try:
        resp = session.get(cal_url, headers=headers, timeout=10)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        day_links = soup.select('a[href*="race_list.html?kaisai_date="]')
        
        dates = set()
        for link in day_links:
            href = link.get('href')
            m = re.search(r'kaisai_date=(\d{8})', href)
            if m:
                dates.add(m.group(1))
        
        return sorted(list(dates))
    except Exception as e:
        print(f"  ⚠️  Error fetching calendar {year}/{month}: {e}")
        return []

def get_race_ids_from_date(date_str, mode='JRA'):
    """
    特定日のレースIDを取得
    
    重要: race_list.htmlではなく、race_list_sub.htmlを使用します。
    メインページはJavaScriptで動的に読み込むため、race_idが含まれていません。
    race_list_sub.htmlには全てのレースリンクが含まれています。
    """
    base_domain = "race.netkeiba.com" if mode == 'JRA' else "nar.netkeiba.com"
    # 🔑 重要: race_list_sub.html を使用
    sub_url = f"https://{base_domain}/top/race_list_sub.html?kaisai_date={date_str}"
    
    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    
    try:
        time.sleep(0.3)  # レート制限対策
        resp = session.get(sub_url, headers=headers, timeout=10)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # race_idを含むリンクを探す
        race_links = soup.find_all('a', href=re.compile(r'race_id=\d+'))
        
        race_ids = set()
        for link in race_links:
            href = link.get('href')
            m = re.search(r'race_id=(\d+)', href)
            if m:
                rid = m.group(1)
                if len(rid) == 12:  # レースIDは12桁
                    race_ids.add(rid)
        
        return sorted(list(race_ids))
        
    except Exception as e:
        print(f"    ⚠️  Error fetching {date_str}: {e}")
        return []

def fetch_race_ids(mode='JRA', start_year=2025, end_year=2026):
    """指定期間のレースIDを全て取得"""
    print(f"\n🚀 {mode} Race ID Fetching ({start_year}-{end_year})...")
    
    save_csv = JRA_ID_CSV if mode == 'JRA' else NAR_ID_CSV
    save_path = os.path.join(DATA_DIR, save_csv)
    
    all_ids = set()
    
    # 既存のIDを読み込み
    if os.path.exists(save_path):
        try:
            existing_df = pd.read_csv(save_path, dtype=str)
            if 'race_id' in existing_df.columns:
                all_ids = set(existing_df['race_id'].dropna().unique())
                print(f"  📂 Loaded {len(all_ids)} existing IDs from {save_csv}")
        except:
            pass
    
    # 年月ごとに処理
    for year in range(start_year, end_year + 1):
        print(f"  📅 Processing {year}...")
        
        for month in range(1, 13):
            # カレンダーから開催日を取得
            dates = get_kaisai_dates(year, month, mode)
            
            if not dates:
                continue
            
            # 各開催日のレースIDを取得
            for date_str in tqdm(dates, desc=f"{year}/{month:02}", leave=False):
                day_ids = get_race_ids_from_date(date_str, mode)
                all_ids.update(day_ids)
                
                # 定期的に保存(100件ごと)
                if len(all_ids) % 100 == 0 and len(all_ids) > 0:
                    df_temp = pd.DataFrame({'race_id': sorted(list(all_ids))})
                    df_temp.to_csv(save_path, index=False)
    
    # 最終保存
    print(f"\n💾 Saving {len(all_ids)} IDs to {save_path}...")
    df_out = pd.DataFrame({'race_id': sorted(list(all_ids))})
    df_out.to_csv(save_path, index=False)
    print("✅ Done.")
    
    return len(all_ids)


In [ ]:
# 実行
if os.path.exists(DATA_DIR):
    try:
        jra_count = fetch_race_ids(mode='JRA', start_year=START_YEAR, end_year=END_YEAR)
        print(f"\n✅ JRA: {jra_count} IDs collected")
    except Exception as e:
        print(f"❌ JRA Error: {e}")
        import traceback
        traceback.print_exc()
    
    try:
        nar_count = fetch_race_ids(mode='NAR', start_year=START_YEAR, end_year=END_YEAR)
        print(f"\n✅ NAR: {nar_count} IDs collected")
    except Exception as e:
        print(f"❌ NAR Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"❌ ディレクトリが見つかりません: {DATA_DIR}")